In [13]:
%%writefile modular/betting_odds_pull.py
import requests
import pandas as pd

# Your API key
api_key = ''

# Define the base URL for The Odds API and the list of player prop markets
base_url = 'https://api.the-odds-api.com/v4/sports'
nba_player_prop_markets = [
    'player_points', 'player_rebounds', 'player_assists',
    'player_threes', 'player_blocks', 'player_steals',
    'player_blocks_steals', 'player_turnovers',
    'player_points_rebounds_assists', 'player_points_rebounds',
    'player_points_assists', 'player_rebounds_assists',
    'player_first_basket', 'player_double_double',
    'player_triple_double', 'player_points_alternate',
    'player_rebounds_alternate', 'player_assists_alternate',
    'player_blocks_alternate', 'player_steals_alternate',
    'player_threes_alternate', 'player_points_assists_alternate',
    'player_points_rebounds_alternate', 'player_rebounds_assists_alternate',
    'player_points_rebounds_assists_alternate'
]

# Initialize a list to store data for each game
all_games_data = []

# Fetch the list of NBA games
games_url = f"{base_url}/basketball_nba/events"
games_params = {'apiKey': api_key}
games_response = requests.get(games_url, params=games_params)

if games_response.status_code == 200:
    games_data = games_response.json()
    
    # Iterate over each game to fetch player props odds
    for game in games_data:
        game_id = game['id']
        
        # Initialize a dictionary to store odds data for this game
        game_odds_data = {
            'Game ID': game_id,
            'Home Team': game['home_team'],
            'Away Team': game['away_team'],
            'Player Props': []
        }
        
        # Fetch odds for player props for this game
        for market in nba_player_prop_markets:
            odds_url = f"{base_url}/basketball_nba/events/{game_id}/odds"
            odds_params = {
                'apiKey': api_key,
                'markets': market,  # Fetch odds for the current market
                'regions': 'us'  # Specify your desired regions
            }
            odds_response = requests.get(odds_url, params=odds_params)
            
            if odds_response.status_code == 200:
                odds_data = odds_response.json()
                # Append the odds data for this market to the game_odds_data
                game_odds_data['Player Props'].append(odds_data)
        
        # Append the collected odds data for this game to the all_games_data list
        all_games_data.append(game_odds_data)

# Convert the list of game data into a pandas DataFrame
df_games = pd.DataFrame(all_games_data)




In [50]:
import requests
import pandas as pd

# Your API key
api_key = ''

# Define the base URL for The Odds API and the list of player prop markets
base_url = 'https://api.the-odds-api.com/v4/sports'
nba_player_prop_markets = [
    'player_points', 'player_rebounds', 'player_assists',
    'player_threes', 'player_blocks', 'player_steals',
    'player_blocks_steals', 'player_turnovers',
    'player_points_rebounds_assists', 'player_points_rebounds',
    'player_points_assists', 'player_rebounds_assists',
    'player_first_basket', 'player_double_double',
    'player_triple_double', 'player_points_alternate',
    'player_rebounds_alternate', 'player_assists_alternate',
    'player_blocks_alternate', 'player_steals_alternate',
    'player_threes_alternate', 'player_points_assists_alternate',
    'player_points_rebounds_alternate', 'player_rebounds_assists_alternate',
    'player_points_rebounds_assists_alternate'
]

# Initialize a list to store data for the games
games_data_list = []

# Fetch the list of NBA games
games_url = f"{base_url}/basketball_nba/events"
games_params = {
    'apiKey': api_key,
    'regions': 'us'
}
games_response = requests.get(games_url, params=games_params)

if games_response.status_code == 200:
    games_data = games_response.json()
    
    # Process data for all available games
    for game in games_data:
        game_id = game['id']
        
        # Initialize a dictionary for this game
        game_odds_data = {
            'Game ID': game_id,
            'Home Team': game['home_team'],
            'Away Team': game['away_team'],
            'Player Props': []
        }
        
        # Fetch odds for player props for this game
        for market in nba_player_prop_markets:
            odds_url = f"{base_url}/basketball_nba/events/{game_id}/odds"
            odds_params = {
                'apiKey': api_key,
                'markets': market,
                'regions': 'us'
            }
            odds_response = requests.get(odds_url, params=odds_params)
            
            if odds_response.status_code == 200:
                odds_data = odds_response.json()
                # Append the odds data for this market to the game_odds_data
                game_odds_data['Player Props'].append(odds_data)
        
        # Append the collected odds data for this game to the games_data_list
        games_data_list.append(game_odds_data)

# Convert the list of game data into a pandas DataFrame
df_games = pd.DataFrame(games_data_list)

# df_games now contains data for all games returned by the API


In [40]:
import pandas as pd
import ast
import json

print(df_games.columns)

# Assuming `df_games` is a DataFrame loaded from your CSV or constructed with the provided data

# Convert the 'Player Props' column from string to actual Python objects
#df_games['Player Props'] = df_games['Player Props'].apply(lambda x: json.loads(x))


# Initialize an empty list to collect data
collected_data = []

# Iterate over each row in the DataFrame to process player props
for index, row in df_games.iterrows():
    game_id = row['Game ID']
    home_team = row['Home Team']
    away_team = row['Away Team']
    
    # Iterate over each player prop in 'Player Props' column
    for player_prop in row['Player Props']:
        # Assuming each 'player_prop' is a dictionary containing 'bookmakers' and other details
        for bookmaker in player_prop.get('bookmakers', []):
            for market in bookmaker.get('markets', []):
                market_key = market.get('key')
                last_update = market.get('last_update', '')
                for outcome in market.get('outcomes', []):
                    player_name = outcome.get('description', '')
                    over_under = outcome.get('name', '')
                    price = outcome.get('price')
                    point = outcome.get('point', '')
                    commence_time = outcome.get('commence_time', '')


                    collected_data.append({
                        'Game ID': game_id,
                        'Home Team': home_team,
                        'Away Team': away_team,
                        'Player Name': player_name,
                        'Market': market_key,
                        'Over/Under': over_under,
                        'Price': price,
                        'Point': point,
                        'last_update': last_update,
                        'commence_time': commence_time
                    })

# Convert the collected data into a DataFrame
df_processed = pd.DataFrame(collected_data)

print(df_processed['commence_time'])

#filter for only Brandon Miller
#df_processed = df_processed[df_processed['Player Name'] == 'Anfernee Simons']


# Convert 'last_update' (format: 2024-02-07T15:39:22Z) by replacing 'T' with a space and removing the 'Z' at the end
df_processed['last_update'] = df_processed['last_update'].str.replace('T', ' ').str.replace('Z', '')
df_processed['last_update'] = pd.to_datetime(df_processed['last_update'])
#print(df_processed['last_update'])
# Sort by 'Player Name', 'Market', 'Over/Under', and 'last_update' in descending order to bring the latest updates to the top
df_processed.sort_values(by=['Player Name', 'last_update'], ascending=[True, False], inplace=True) #'Market', 'Over/Under',
#print(df_processed)

# Drop duplicates, keeping only the first (most recent) entry for each combination of 'Player Name', 'Market', and 'Over/Under'
df_latest_odds = df_processed.drop_duplicates(subset=['Player Name', 'Market', 'Over/Under', 'last_update'], keep='first')

#filter for Over/Under as 'Yes'
#df_latest_odds = df_latest_odds[df_latest_odds['Over/Under'] == 'Yes']

# Create separate columns for Over Price and Under Price
df_latest_odds['Over Price'] = df_latest_odds.apply(lambda x: x['Price'] if x['Over/Under'] == 'Over' else None, axis=1)
df_latest_odds['Under Price'] = df_latest_odds.apply(lambda x: x['Price'] if x['Over/Under'] == 'Under' else None, axis=1)

# Pivot the table for points (points are the same for both Over and Under)
df_pivoted_points = df_latest_odds.pivot_table(
    index=['Game ID', 'Home Team', 'Away Team', 'Player Name'],
    columns='Market',
    values='Point',
    aggfunc='first'
).reset_index()

# Pivot the table for prices
df_pivoted_prices = df_latest_odds.pivot_table(
    index=['Game ID', 'Home Team', 'Away Team', 'Player Name'],
    columns='Market',
    values=['Over Price', 'Under Price'],
    aggfunc='first'
).reset_index()

# Flatten MultiIndex for prices
df_pivoted_prices.columns = [' '.join(col).strip() for col in df_pivoted_prices.columns.values]

# Merge points and prices DataFrames
odds_df = pd.merge(df_pivoted_points, df_pivoted_prices, on=['Game ID', 'Home Team', 'Away Team', 'Player Name'], how='left')

#print(odds_df.head())

#pull out player's without any odds
odds_df = odds_df.dropna(subset=['player_points'])
#print(odds_df.head())

#count nulls in each column
#print(odds_df.isnull().sum())
#print(len(odds_df['Player Name'].unique()))

# Check for duplicates based on 'Player Name'
player_counts = odds_df['Player Name'].value_counts()
# Identify if there are duplicate players
duplicate_players = player_counts[player_counts > 1]

# Print out the result
# print("Duplicate Players:")
# print(duplicate_players)



Index(['Game ID', 'Home Team', 'Away Team', 'Player Props'], dtype='object')
0        
1        
2        
3        
4        
       ..
3494     
3495     
3496     
3497     
3498     
Name: commence_time, Length: 3499, dtype: object


C:\Users\ghadf\AppData\Local\Temp\ipykernel_25136\3229987810.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest_odds['Over Price'] = df_latest_odds.apply(lambda x: x['Price'] if x['Over/Under'] == 'Over' else None, axis=1)
C:\Users\ghadf\AppData\Local\Temp\ipykernel_25136\3229987810.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest_odds['Under Price'] = df_latest_odds.apply(lambda x: x['Price'] if x['Over/Under'] == 'Under' else None, axis=1)


Pull out the season stats

In [1]:
%%writefile modular/player_game_logs.py
import pandas as pd
from datetime import datetime, timedelta
from nba_api.stats.endpoints import commonallplayers, playergamelog, leaguedashplayerstats, leaguegamefinder
from nba_api.stats.static import teams
import time
import numpy as np


def get_current_nba_season_year():
    current_date = datetime.now()
    if current_date.month > 9:  # NBA season starts in October
        return str(current_date.year) + "-" + str(current_date.year + 1)[2:]
    else:
        return str(current_date.year - 1) + "-" + str(current_date.year)[2:]

def calculate_cumulative_win_rates(season):
    try:
        # Adjust the season start date based on the typical NBA season start dates
        season_start_date = season.split('-')[0] + "-10-01"  # Assuming October 1st as a generic start date
        all_games = leaguegamefinder.LeagueGameFinder(season_nullable=season).get_data_frames()[0]
        all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE'])
        all_games = all_games[all_games['GAME_DATE'] > pd.to_datetime(season_start_date)]
        all_games = all_games.sort_values('GAME_DATE')
        all_games['WIN'] = all_games['WL'].apply(lambda x: 1 if x == 'W' else 0)
        all_games['CUMULATIVE_WINS'] = all_games.groupby('TEAM_NAME')['WIN'].cumsum()
        all_games['CUMULATIVE_GAMES'] = all_games.groupby('TEAM_NAME').cumcount() + 1
        all_games['CUMULATIVE_WIN_RATE'] = all_games['CUMULATIVE_WINS'] / all_games['CUMULATIVE_GAMES']
        return all_games
    except Exception as e:
        print(f"Error calculating cumulative win rates: {e}")
        return pd.DataFrame()
    

def get_win_rate(row, team_type, all_games):
    game_date = row['GAME_DATE']
    team_name = row[team_type]
    team_games = all_games[(all_games['TEAM_NAME'] == team_name) & (all_games['GAME_DATE'] < game_date)]
    if not team_games.empty:
        return team_games.iloc[-1]['CUMULATIVE_WIN_RATE']
    else:
        return 0.0


def load_nba_player_game_logs(seasons, min_avg_minutes=30.0, save_path='data/player_game_logs.csv'):
    if not isinstance(seasons, list):
        seasons = [seasons]

    new_players_data = pd.DataFrame()

    for season in seasons:
        print(f"Processing season {season}...")
        try:
            all_players = commonallplayers.CommonAllPlayers(is_only_current_season=0).get_data_frames()[0]
            player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season).get_data_frames()[0]
        except Exception as e:
            print(f"Error fetching player stats for season {season}: {e}")
            continue

        # Filter for players who meet the minimum average minutes threshold
        player_stats['AVG_MIN'] = player_stats['MIN'] / player_stats['GP']
        eligible_players = player_stats[player_stats['AVG_MIN'] >= min_avg_minutes]
        teams_list = teams.get_teams()
        team_abbrev_to_full_name = {team['abbreviation']: team['full_name'] for team in teams_list}

        all_games = calculate_cumulative_win_rates(season)
        if all_games.empty:
            print("Skipping win rate calculation due to an error.")
            continue

        for index, player in eligible_players.iterrows():
            try:
                player_id = player['PLAYER_ID']
                player_name = player['PLAYER_NAME']
                player_log = playergamelog.PlayerGameLog(player_id=player_id, season=season)
                player_data = player_log.get_data_frames()[0]
                if player_data.empty:
                    continue
                player_data['PLAYER_NAME'] = player_name
                player_data['TEAM_ABBREVIATION'] = player_data['MATCHUP'].str[:3]
                player_data['OPPONENT_ABBREVIATION'] = player_data['MATCHUP'].apply(lambda x: x.split(' ')[2] if 'vs.' in x else x.split(' ')[-1])
                player_data['TEAM_NAME'] = player_data['TEAM_ABBREVIATION'].map(team_abbrev_to_full_name)
                player_data['OPPONENT_NAME'] = player_data['OPPONENT_ABBREVIATION'].map(team_abbrev_to_full_name)
                player_data = player_data[player_data['TEAM_ABBREVIATION'].isin(team_abbrev_to_full_name.keys())]
                new_players_data = pd.concat([new_players_data, player_data], ignore_index=True)
            except Exception as e:
                print(f"Error processing player {player_name}: {e}")
                continue
            time.sleep(0.6)  # To respect rate limits

        # Calculate team and opponent win rates
        new_players_data['GAME_DATE'] = pd.to_datetime(new_players_data['GAME_DATE'])
        new_players_data['TEAM_WIN_RATE'] = new_players_data.apply(lambda row: get_win_rate(row, 'TEAM_NAME', all_games), axis=1)
        new_players_data['OPPONENT_WIN_RATE'] = new_players_data.apply(lambda row: get_win_rate(row, 'OPPONENT_NAME', all_games), axis=1)

        new_players_data['home_away'] = new_players_data['MATCHUP'].str.split(' ').str[1]
        new_players_data['home_away'] = new_players_data['home_away'].apply(lambda x: 'Away' if '@' in x else 'Home')
        new_players_data.reset_index(drop=True)
    if not new_players_data.empty:
        new_players_data.to_csv(save_path, index=False)
        print(f"Player game logs saved to {save_path}")
    else:
        print("No player game logs to save.")



# Example usage
#seasons = ['2022-23']  # You can adjust seasons as needed
#load_nba_player_game_logs(seasons, min_avg_minutes=30.0, save_path='data/player_game_logs_winr.csv')
        
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nba_api.stats.static import teams

def prepare_upcoming_games_data(season_games_csv, player_game_logs_csv, expand_with_players=False):
    # Load season games data
    data = pd.read_csv(season_games_csv)
    
    # Process home and away data
    home_data = data[['DATE', 'Start (ET)', 'Home/Neutral']].copy()
    home_data['Home_Away'] = 'Home'
    home_data['MATCHUP'] = home_data['Home/Neutral'] + ' vs. ' + data['Visitor/Neutral']
    home_data.rename(columns={'Home/Neutral': 'Team'}, inplace=True)
    home_data['WL_encoded'] = np.nan
    
    away_data = data[['DATE', 'Start (ET)', 'Visitor/Neutral']].copy()
    away_data['Home_Away'] = 'Away'
    away_data['MATCHUP'] = away_data['Visitor/Neutral'] + ' @ ' + home_data['Team']  # Adjusted to use '@' for away games
    away_data.rename(columns={'Visitor/Neutral': 'Team'}, inplace=True)
    away_data['WL_encoded'] = np.nan
    
    final_data = pd.concat([home_data, away_data], ignore_index=True)
    final_data.sort_values(by=['DATE', 'Start (ET)', 'Home_Away'], inplace=True)
    final_data.reset_index(drop=True, inplace=True)
    
    # Convert 'DATE' column to datetime format
    final_data['DATE'] = pd.to_datetime(final_data['DATE'], format='%a, %b %d, %Y')
    
    # Get unique team information from the NBA API
    teams_info = teams.get_teams()
    teams_df = pd.DataFrame(teams_info)
    teams_df.rename(columns={'id': 'TEAM_ID', 'full_name': 'TEAM_NAME', 'abbreviation': 'TEAM_ABBREVIATION'}, inplace=True)
    
    # Merge final_data with teams_df to include TEAM_ID and abbreviations
    final_data = pd.merge(final_data, teams_df[['TEAM_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION']], left_on='Team', right_on='TEAM_NAME', how='left')
    
    # Ensure all team names in MATCHUP are abbreviations
    for index, row in final_data.iterrows():
        for _, team_row in teams_df.iterrows():
            final_data.at[index, 'MATCHUP'] = final_data.at[index, 'MATCHUP'].replace(team_row['TEAM_NAME'], team_row['TEAM_ABBREVIATION'])
    
    # Extract and filter for upcoming games
    today = pd.Timestamp.now().floor('D')  # Normalize to avoid time part
    week_out = today + timedelta(days=7)
    upcoming_games = final_data[(final_data['DATE'] >= today) & (final_data['DATE'] <= week_out)]
    upcoming_games.sort_values(by='DATE', inplace=True)
    upcoming_games.reset_index(drop=True, inplace=True)
    
    # Format the 'DATE' column to match the example output's 'GAME_DATE' format
    upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
    
    # Correct the column name for consistency
    upcoming_games.rename(columns={'Home_Away': 'home_away'}, inplace=True)
    
    # Drop unnecessary columns and adjust to match the target dataset structure
    upcoming_games = upcoming_games[['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_ID', 'TEAM_NAME']]
    
    # Create OPPOSING_TEAM column
    upcoming_games['OPPONENT_NAME'] = np.nan  # Placeholder for opposing team names
    
    # Populate TEAM_NAME and OPPOSING_TEAM with correct names
    for index, row in upcoming_games.iterrows():
        if row['home_away'] == 'Home':
            # If it's a home game, the home team is TEAM_NAME and the visitor team is OPPOSING_TEAM
            home_team_abbr = row['MATCHUP'].split(' vs. ')[0]
            away_team_abbr = row['MATCHUP'].split(' vs. ')[1]
        else:
            # If it's an away game, the visitor team is TEAM_NAME and the home team is OPPOSING_TEAM
            away_team_abbr = row['MATCHUP'].split(' @ ')[0]
            home_team_abbr = row['MATCHUP'].split(' @ ')[1]

        home_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == home_team_abbr]['TEAM_NAME'].values[0]
        away_team_full_name = teams_df[teams_df['TEAM_ABBREVIATION'] == away_team_abbr]['TEAM_NAME'].values[0]

        upcoming_games.at[index, 'TEAM_NAME'] = home_team_full_name if row['home_away'] == 'Home' else away_team_full_name
        upcoming_games.at[index, 'OPPONENT_NAME'] = away_team_full_name if row['home_away'] == 'Home' else home_team_full_name


    # Load player game logs to use for fetching rosters
    player_game_logs = pd.read_csv(player_game_logs_csv)
    print(player_game_logs.columns)
    print(upcoming_games.columns)
    
    if expand_with_players:
        expanded_games_with_players = pd.DataFrame()

        # Assuming player_game_logs_csv is correctly loaded into player_game_logs DataFrame
        player_game_logs = pd.read_csv('data\player_game_logs_winr.csv')

        expanded_rows = []

        for _, game in upcoming_games.iterrows():
            team_name = game['TEAM_NAME']
            team_players = player_game_logs[player_game_logs['TEAM_NAME'] == team_name]

            for _, player in team_players.iterrows():
                expanded_row = game.copy().to_dict()
                expanded_row['Player_ID'] = player['Player_ID']
                expanded_row['PLAYER_NAME'] = player['PLAYER_NAME']
                expanded_rows.append(expanded_row)

        expanded_games_with_players = pd.DataFrame(expanded_rows)

        #drop duplicate players and game_dates
        expanded_games_with_players = expanded_games_with_players.drop_duplicates(subset=['GAME_DATE', 'PLAYER_NAME'], keep='first')

        # only include these columns: ['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_NAME','OPPOSING_TEAM', 'Player_ID', 'PLAYER_NAME']
        expanded_games_with_players = expanded_games_with_players[['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_NAME', 'OPPONENT_NAME', 'Player_ID', 'PLAYER_NAME']]
        
        # Return the expanded DataFrame
        return expanded_games_with_players


    return upcoming_games

# Example usage with file paths
#season_games_csv = 'data/23_24_season_games.csv'
#player_game_logs_csv = 'data/player_game_logs_winr.csv'
#upcoming_games_df = prepare_upcoming_games_data(season_games_csv, player_game_logs_csv, expand_with_players=True)
#print(upcoming_games_df.head())
#print(len(upcoming_games_df))


Overwriting modular/player_game_logs.py


In [2]:
%%writefile modular/metrics_functions.py
import pandas as pd
import numpy as np

def calculate_running_stats(group, stats):
    """
    Calculate running mean and standard deviation for the specified statistics.
    """
    # Calculate mean and std for the last n games
    running_means = group[stats].mean()
    running_stds = group[stats].std(ddof=0)  # ddof=0 for population standard deviation
    return running_means, running_stds

def prepare_mean_std_data(df, n_games=10, current_date=None, current_season=None, game_location='All'):
    """
    Prepare aggregated data for players over the last n games up to the current date and within the current season,
    considering home/away context.
    """
    stats = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'AST', 'OREB', 'DREB', 'REB', 'TOV', 'STL', 'BLK', 'MIN', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE']
    
    if current_date:
        df = df[df['GAME_DATE'] <= current_date]
    if current_season:
        df = df[df['SEASON'] == current_season]
    if game_location in ['Home', 'Away']:
        df = df[df['home_away'] == game_location]

    grouped = df.groupby(['PLAYER_NAME', 'TEAM_NAME'])
    result_list = []

    for (player_name, team_name), group in grouped:
        group = group.sort_values(by='GAME_DATE', ascending=False).head(n_games)
        mean_values, std_values = calculate_running_stats(group, stats)
        
        mean_values['TYPE'] = 'mean_'+str(n_games)+'_games'
        std_values['TYPE'] = 'std_'+str(n_games)+'_games'
        mean_values['PLAYER_NAME'] = player_name
        mean_values['TEAM_NAME'] = team_name
        mean_values['home_away'] = game_location
        
        std_values['PLAYER_NAME'] = player_name
        std_values['TEAM_NAME'] = team_name
        std_values['home_away'] = game_location

        result_list.append(mean_values)
        result_list.append(std_values)

    result_df = pd.DataFrame(result_list).reset_index(drop=True)
    return result_df


# Example usage
# Load in data
data = pd.read_csv('data/player_game_logs_winr.csv')
# Filter for a specific player, e.g., Cade Cunningham
data = data[data['PLAYER_NAME'] == 'Cade Cunningham']
# Assuming 'data' is your DataFrame loaded from 'player_game_logs_winr.csv'
aggregated_data = prepare_mean_std_data(data, n_games=10, game_location='Home')
#print(aggregated_data.head())
#print(aggregated_data.columns)

def calculate_league_stats(df, stats, n_games=10, current_date=None, current_season=None, game_location='All'):
    """
    Calculate league-wide standard deviation for the specified statistics over the last n games,
    considering the filters applied for date, season, and location.
    """
    if current_date:
        df = df[df['GAME_DATE'] <= current_date]
    if current_season:
        df = df[df['SEASON'] == current_season]
    if game_location in ['Home', 'Away']:
        df = df[df['home_away'] == game_location]

    std_values = df[stats].std(ddof=0)  # Using population standard deviation

    # Correctly creating a DataFrame with the intended structure
    std_values_df = pd.DataFrame([std_values.values], columns=stats)  # Wrap in a list to create a single row DataFrame
    std_values_df['TYPE'] = 'league_std_' + str(n_games) + '_games'
    std_values_df['PLAYER_NAME'] = 'League'
    std_values_df['TEAM_NAME'] = 'All'
    std_values_df['home_away'] = game_location

    return std_values_df


def prepare_league_std_data(df, n_games=10, current_date=None, current_season=None, game_location='All'):
    """
    Prepare league-wide aggregated standard deviation data over the last n games up to the current date and within the current season,
    considering home/away context.
    """
    stats = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'AST', 'OREB', 'DREB', 'REB', 'TOV', 'STL', 'BLK', 'MIN', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE']
    league_stats = calculate_league_stats(df, stats, n_games, current_date, current_season, game_location)
    
    result_df = pd.DataFrame(league_stats).reset_index(drop=True)
    return result_df

# Example usage
# Assuming 'data' is your DataFrame loaded from 'player_game_logs_winr.csv'
#league_data = prepare_league_std_data(data, n_games=10, game_location='Home')
#print(league_data.head())

#example concatenated data
# Concatenate league_data and aggregated_data
#combined_data = pd.concat([league_data, aggregated_data], ignore_index=True)

# Check the first few rows of the combined dataframe to ensure it looks correct
#print(combined_data.head())

# Optionally, check the structure and summary of the combined dataframe
#print(combined_data.info())


def prepare_performance_against_all_teams(df):
    """
    Prepare aggregated data for each player against each team they've played against in the dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing player game logs.
    
    Returns:
    - DataFrame: The aggregated data with running averages for each player against each team.
    """
    stats = ['PTS', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'AST', 'OREB', 'DREB', 'REB', 'TOV', 'STL', 'BLK', 'MIN', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE']
    unique_players = df['PLAYER_NAME'].unique()
    unique_teams = df['OPPONENT_NAME'].unique()
    
    result_list = []

    for player in unique_players:
        for team in unique_teams:
            player_games = df[(df['PLAYER_NAME'] == player) & (df['OPPONENT_NAME'] == team)]
            
            if not player_games.empty:
                mean_values, _ = calculate_running_stats(player_games, stats)
                mean_values['PLAYER_NAME'] = player
                mean_values['OPPONENT_NAME'] = team
                mean_values['TYPE'] = f'mean_vs_{team}'
                result_list.append(mean_values)
    
    result_df = pd.DataFrame(result_list).reset_index(drop=True)
    return result_df

#Example usage
#performance_against_all_teams = prepare_performance_against_all_teams(data)
#print(performance_against_all_teams)




Overwriting modular/metrics_functions.py


In [41]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from modular.player_game_logs import load_nba_player_game_logs, prepare_upcoming_games_data
from modular.metrics_functions import prepare_mean_std_data, prepare_league_std_data, prepare_performance_against_all_teams
from modular.betting_functions import calculate_probability, calculate_bet_outcome, generate_betting_options, evaluate_bets, evaluate_bets_n_games_debug

# Add a new section in your sidebar for navigation
st.sidebar.header("Navigation")
page = st.sidebar.radio("Select a page:", ["Player Analysis", "Forecasting Player Statistics"])


#------------Loading data with caching---------------
st.sidebar.header("Refresh Data for Select Season Year with Players at the Minimum Average Minutes Played")
# Season Year Selection slider
season_years = ['2022-23', '2023-24', '2024-25']  # Update this list with available seasons
selected_season = st.sidebar.selectbox('Select Season Year', season_years)

# min avg selection
min_avg_minutes =st.sidebar.slider('Minimum Average Minutes Played', min_value=1, max_value=60, value=20, step=1)

# Option to reload data
if st.sidebar.button('Load/Refresh Data'):
    load_nba_player_game_logs([selected_season], min_avg_minutes=min_avg_minutes, save_path='data/player_game_logs_winr.csv')
    st.sidebar.success(f"Data for the {selected_season} season loaded successfully.")

# Loading data with caching
@st.cache(ttl=3600, max_entries=10, show_spinner=False)
def load_data():
    data = pd.read_csv('data/player_game_logs_winr.csv')
    data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'])
    data.sort_values(by='GAME_DATE', inplace=True)
    return data

#pull in upcoming games to concatenate to data and input averages onto it
upcoming_games = prepare_upcoming_games_data('data/23_24_season_games.csv', 'data/player_game_logs_winr.csv', expand_with_players=True)

# Load the existing games data
previous_games = load_data()

# Ensure GAME_DATE is in datetime format for comparison
upcoming_games['GAME_DATE'] = pd.to_datetime(upcoming_games['GAME_DATE'])

# Filter out upcoming games that have dates already in previous games
unique_upcoming_games = upcoming_games[~upcoming_games['GAME_DATE'].isin(previous_games['GAME_DATE'])]

# Concatenate the unique upcoming games to the previous games dataset
data = pd.concat([previous_games, unique_upcoming_games], ignore_index=True)

# Sort the concatenated data by GAME_DATE to maintain chronological order
data.sort_values(by='GAME_DATE', inplace=True)

# Reset the index of the concatenated DataFrame
data.reset_index(drop=True, inplace=True)
#------------Loading data with caching---------------

# Use if-else to control the page display based on the sidebar selection
if page == "Player Analysis":
    # Title of the app
    st.title("NBA Player Game Logs and Statistical Insights")

    # Sidebar for user inputs
    st.sidebar.header("User Input Features")

    #------------Date/Player data---------------


    # Unique Data
    unique_dates = data['GAME_DATE'].dt.strftime('%Y-%m-%d').unique()
    #players = data['PLAYER_NAME'].unique()
    games = data['MATCHUP'].unique()

    # App functions
    # --------Date Selection----------
    selected_date = st.sidebar.selectbox('Select a Date', unique_dates)
    # to provide data for the selected date
    current_data = data[data['GAME_DATE'] == pd.to_datetime(selected_date)]
    players = current_data['PLAYER_NAME'].unique()
    # --------Player Search Selection----------
    # Use a text input for search instead of a dropdown
    search_query = st.sidebar.text_input("Search Player Name")
    # Filter the list of players based on the search query
    filtered_players = [player for player in players if search_query.lower() in player.lower()]
    # If there are too many matches, you might want to limit the number displayed or adjust UI accordingly
    if len(filtered_players) > 1:
        st.sidebar.write("Please refine your search to see the results")
    # --------Player Dropdown Selection----------
    # Allow the user to select a player from the filtered list
    selected_player = st.sidebar.selectbox("Select a Player", filtered_players)

    # Datasets to use
    current_data = data[data['GAME_DATE'] == pd.to_datetime(selected_date)]
    player_data = data[data['PLAYER_NAME'] == selected_player]
    player_date_data = player_data[player_data['GAME_DATE'] == pd.to_datetime(selected_date)]
    current_stats_data = data[data['GAME_DATE'] <= selected_date]
    total_games_played = current_stats_data[current_stats_data['PLAYER_NAME'] == selected_player].shape[0]


    if not player_data.empty:
        game_location = 'Home' if player_data['home_away'].iloc[0] == 'Home' else 'Away'
        game_opposing_team = player_data['OPPONENT_NAME'].iloc[0]
        st.write(f"Data for {selected_player} ({game_location} game) against {game_opposing_team} on {selected_date}:")
        #st.dataframe(player_data[['GAME_DATE', 'TEAM_NAME', 'home_away', 'PLAYER_NAME', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE', 'PTS', 'FG3M', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'MIN']])
    else:
        st.write(f"No data available for {selected_player} on {selected_date}.")


    # Filter data for the selected date
    current_stats_data = data[data['GAME_DATE'] <= selected_date]
    # Total games played by the player in the dataset
    total_games_played = current_stats_data[current_stats_data['PLAYER_NAME'] == selected_player].shape[0]
    # Display total games played
    print(f"Total games played by {selected_player} in the dataset: {total_games_played}")

    # Computing averages and league standard deviation
    # (Ensure functions like prepare_mean_std_data and prepare_league_std_data are correctly implemented)
    total_averages_data = prepare_mean_std_data(current_stats_data, n_games=total_games_played, game_location='All') #, current_date=current_date
    n_game_aggregated_data_all = prepare_mean_std_data(current_stats_data, n_games=10, game_location='All') #, current_date=current_date
    n_game_aggregated_data_home_or_away = prepare_mean_std_data(current_stats_data, n_games=10, game_location=game_location) #, current_date=current_date
    league_std_data = prepare_league_std_data(current_stats_data, n_games=10, game_location=game_location)

    # Performance against all teams
    performance_against_all_teams = prepare_performance_against_all_teams(current_stats_data)
    #filter against opposing_team
    performance_against_all_teams = performance_against_all_teams[performance_against_all_teams['OPPONENT_NAME'] == game_opposing_team]

    performance_against_all_teams = performance_against_all_teams.drop(columns=['OPPONENT_NAME'])
    #print(performance_against_all_teams.head())

    # Concatenate data for analysis
    combined_data = pd.concat([total_averages_data, n_game_aggregated_data_all, n_game_aggregated_data_home_or_away, league_std_data, performance_against_all_teams], ignore_index=True)

    # Filter for selected player and league standard
    combined_data_filtered = combined_data[combined_data['PLAYER_NAME'].isin([selected_player, 'League'])]
    #print(combined_data_filtered[['PLAYER_NAME', 'PTS', 'TYPE', 'home_away', 'TEAM_NAME']])

    st.write(f"Total games played by {selected_player} in the dataset: {total_games_played}")
    st.dataframe(combined_data_filtered[['TEAM_NAME', 'home_away', 'PLAYER_NAME', 'TYPE', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE', 'PTS', 'FG3M', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'MIN']])

    # Move Statistic Selection to Main Body
    stats_options = ['PTS', 'REB', 'AST', 'STL', 'BLK']  # Extend with more stats as needed
    selected_stat = st.selectbox('Select a Statistic for Graph', stats_options)

    # Graph Visualization
    fig, ax = plt.subplots()
    player_season_data = data[data['PLAYER_NAME'] == selected_player]
    ax.plot(player_season_data['GAME_DATE'], player_season_data[selected_stat], marker='o', linestyle='-', label=selected_stat)
    ax.set_title(f"{selected_stat} Trend for {selected_player} During the Season")
    ax.set_xlabel('Game Date')
    ax.set_ylabel(selected_stat)
    plt.xticks(rotation=45)
    plt.legend()
    st.pyplot(fig)

    # Betting Analysis Section
    st.header("Betting Analysis")

    # Ensure that player_season_data is filtered for the selected player
    player_season_data = combined_data_filtered[combined_data_filtered['PLAYER_NAME'] == selected_player]


    #Testing the generate_betting_options filter-------------------------------------------------------------------------------------

    # Ensure GAME_DATE is in datetime format
    player_data['GAME_DATE'] = pd.to_datetime(player_data['GAME_DATE'])

    # Filter for selected date directly
    selected_date_dt = pd.to_datetime(selected_date)

    # Get today's date as a datetime object at midnight
    today = datetime.now()
    today = pd.to_datetime(today)
    yesterday = today - timedelta(days=1)
    yesterday = pd.to_datetime(yesterday)

    # Filter for selected date
    # If the selected date is today or in the future, filter data up to yesterday
    if selected_date_dt >= today:
        print("Selected date is today or in the future.")
        player_data_filt = player_data[player_data['GAME_DATE'] <= yesterday]
    # If the selected date is before today, filter for the selected day
    else:
        print("Selected date is before today.")
        player_data_filt = player_data[player_data['GAME_DATE'] <= selected_date_dt]

    # print("Filtered player data for selected date:", player_data_filt)

    #Testing the generate_betting_options filter-------------------------------------------------------------------------------------



    # Sidebar for interactive parameters
    n_games = st.sidebar.slider('Number of Games', min_value=1, max_value=60, value=10)
    league_std_rate = st.sidebar.slider('League Standard Deviation Above Rate', min_value=0.0, max_value=1.0, value=0.9, step=0.01)
    probability_high = st.sidebar.slider('High Probability Threshold', min_value=0.0, max_value=1.0, value=0.9, step=0.01)
    probability_low = st.sidebar.slider('Low Probability Threshold', min_value=0.0, max_value=1.0, value=0.1, step=0.01)

    with st.form("betting_form"):
        selected_stat_for_bet = st.selectbox('Select Statistic for Betting', stats_options)
        bet_stat_projection = st.number_input('Enter Bet Stat Projection', value=0.0, format="%.2f")
        bet_amount = st.number_input('Enter Bet Amount ($)', value=0.0, format="%.2f")
        odds = st.number_input('Enter Odds (American format, e.g., +150 or -150)', value=100)
        submit_bet = st.form_submit_button("Calculate")

    # When calling calculate_probability
    if submit_bet and not player_data.empty:
        probability, against_team_probability, number_of_games_against_team, player_std, std_dev_comparison, league_std, number_of_games_above_projection, number_of_games = calculate_probability(
            player_data, selected_stat_for_bet, bet_stat_projection, league_std_data, n_games, league_std_rate, game_opposing_team
        )

        
        # Call calculate_bet_outcome to get expected_profit and expected_loss
        expected_profit, expected_loss, probability_weighted_to_profit = calculate_bet_outcome(bet_amount, odds, probability)

        # Now you can correctly display the expected profit and loss
        st.write(f"Probability of achieving projection: {probability*100:.2f}% out of {number_of_games} games")
        if against_team_probability is not None:
            st.write(f"Probability against {game_opposing_team}: {against_team_probability*100:.2f}% with {number_of_games_against_team} games above projection")
        st.write(f"Player's Std Dev: {player_std:.2f}, Better than league's by 10%: {'Yes' if std_dev_comparison else 'No'}")

        # Your recommendation logic
        if probability >= probability_high and std_dev_comparison:
            recommendation = "Betting above the projection might be more favorable due to high probability and player's consistency."
        elif probability <= probability_low and std_dev_comparison:
            recommendation = "Betting below the projection might be more favorable due to low probability and player's consistency."
        else:
            recommendation = "Consider other options or exercise caution due to lower probability or player's inconsistency."
        st.write(recommendation)

        # Correctly output the expected profit and loss
        st.write(f"Expected Profit if Win: ${expected_profit:.2f}, Expected Loss if Lose: -${expected_loss:.2f}")
        st.write(f"Weighted Probability to Profit: {probability_weighted_to_profit:.2f}")

        st.write(f"Total games played by {selected_player} in the dataset: {total_games_played}")
        st.dataframe(combined_data_filtered[['TEAM_NAME', 'home_away', 'PLAYER_NAME', 'TYPE', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE', 'MIN', selected_stat_for_bet]])

        #print("Preview of player_data:")
        #print(player_data.head())
        #print("Preview of league_std_data:")
        #print(league_std_data.head())

        betting_options_df = generate_betting_options(
            player_data_filt, league_std_data, selected_player, game_opposing_team, 
            all_players=False, n_games=n_games, league_std_rate=league_std_rate, 
            probability_high=probability_high, probability_low=probability_low
        )

        #print("betting_options_df.head()=", betting_options_df.head())
        #filter for Stats
        print("selected stat=", selected_stat_for_bet)
        print("betting option columns =", betting_options_df.columns)
        #print("betting option head =", betting_options_df.head(1))
        #betting_options_df = betting_options_df[betting_options_df['Stat'] == selected_stat_for_bet]
        print("betting option columns =", betting_options_df.columns)
        #print("betting option head =", betting_options_df.head())
        st.dataframe(betting_options_df)

    # Ensure the correct datetime format and sort order
    print("Step 1: Data Preparation Completed")

    # Step 2: Generate Betting Options for all historical data
    betting_options_df = generate_betting_options(
        player_data_filt, league_std_data, selected_player, game_opposing_team, all_players=True, n_games=n_games, league_std_rate=league_std_rate, 
        probability_high=probability_high, probability_low=probability_low)
    print(f"Step 2: Generated {len(betting_options_df)} betting options for all historical data.")
    

    # Step 3: Filter Betting Options for the selected date
    selected_date_dt = pd.to_datetime(selected_date)
    if not betting_options_df.empty:
        betting_options_df_selected_date = betting_options_df[betting_options_df['GAME_DATE'] == selected_date_dt]
        print(f"Step 3: Filtered {len(betting_options_df_selected_date)} betting options for the selected date ({selected_date}):")
        #print(betting_options_df_selected_date[['PLAYER_NAME', 'Stat', 'Threshold', 'GAME_DATE']])
        st.dataframe(betting_options_df_selected_date)

        # Step 4: Evaluate Bets for an overall evaluation based on the last n games
        unique_dates = betting_options_df['GAME_DATE'].unique()
        if len(unique_dates) > n_games:
            unique_dates_n_games = unique_dates[-n_games:]
            betting_options_df_n_games = betting_options_df[betting_options_df['GAME_DATE'].isin(unique_dates_n_games)]
        else:
            betting_options_df_n_games = betting_options_df

        evaluated_bets_df = evaluate_bets(betting_options_df_n_games, player_data_filt)
        print("Step 4: Evaluated bets based on the last {n_games} games.")

        # Debugging prints for evaluated bets
        print(f"Evaluated Bets DataFrame has {len(evaluated_bets_df)} rows after evaluation.")

        #filter out NaN 
        evaluated_bets_df = evaluated_bets_df.dropna(subset=['Bet Outcome'])

        #get min and max
        min_date = evaluated_bets_df['GAME_DATE'].min()
        max_date = evaluated_bets_df['GAME_DATE'].max()

        # Display the last values of Running Correct and Running Incorrect
        if not evaluated_bets_df.empty:
            final_corrects = evaluated_bets_df.iloc[-1]['Running Correct']
            final_incorrects = evaluated_bets_df.iloc[-1]['Running Incorrect']
            overall_bets = final_corrects + final_incorrects
            print(f"Final Corrects: {final_corrects}, Final Incorrects: {final_incorrects}")
        else:
            print("No bets evaluated.")
        
        #Display Percentage of correct bets through Running Correct and Running Incorrect
        if final_corrects + final_incorrects > 0:
            correct_percentage = final_corrects / overall_bets
            st.write(f"Percentage of correct bets: {correct_percentage*100:.2f}% out of {overall_bets} chances over the last {n_games} games ({min_date} to {max_date})")
        else:
            st.write("No bets evaluated yet.")
    else:
        print("No betting options generated yet.")
        st.write("No betting options generated yet.")




#-------------------------------New Tab for All Around Dashboard for Betting--------------------------------
#elif page == "Overall Betting Details":
    #changes:
    #1. add in a team drop down
    #2. ensure these dropdowns go back to normal when nothings selected
    #3. add in parlay betting, so it should save each bet chosen unless user deletes the bets and will add the parlay automatically
    # New section for Betting Details
# ...

# elif page == "Forecasting Player Statistics":
#     st.header("Statistic Details")
#     st.title("NBA Players Forecasted Statistics")

#     # Sidebar selections
#     unique_dates = data['GAME_DATE'].dt.strftime('%Y-%m-%d').unique()
#     selected_date = st.sidebar.selectbox('Select a Date', unique_dates)
#     #select date and filter for the players and teams
#     data = data[data['GAME_DATE'] == pd.to_datetime(selected_date)]
#     selected_players = st.sidebar.multiselect("Select Players", options=data['PLAYER_NAME'].unique())
#     selected_teams = st.sidebar.multiselect("Select Teams", options=data['TEAM_NAME'].unique())  # Assuming this is used somewhere in your app
#     game_location = st.sidebar.selectbox('Select Game Location', ['All', 'Home', 'Away'])

#     # Parameters for betting options
#     n_games = st.sidebar.slider('Number of Games', min_value=1, max_value=60, value=10)
#     league_std_rate = st.sidebar.slider('League Standard Deviation Above Rate', min_value=0.0, max_value=1.0, value=0.9, step=0.01)
#     probability_high = st.sidebar.slider('High Probability Threshold', min_value=0.0, max_value=1.0, value=0.9, step=0.01)
#     probability_low = st.sidebar.slider('Low Probability Threshold', min_value=0.0, max_value=1.0, value=0.1, step=0.01)

#     # Filter data for the selected date
#     betting_today_data = data[data['GAME_DATE'] == pd.to_datetime(selected_date)]
#     league_std_data = prepare_league_std_data(betting_today_data, n_games=n_games, game_location=game_location)

#     if selected_players:
#         evaluated_bets_list = []
#         for player in selected_players:
#             player_data = betting_today_data[betting_today_data['PLAYER_NAME'] == player]
#             evaluated_bets_df = evaluate_bets_n_games_debug(player_data, player_data, n_games)  # Adjust this call as needed
#             evaluated_bets_list.append(evaluated_bets_df)

#         combined_evaluated_bets_df = pd.concat(evaluated_bets_list, ignore_index=True) if evaluated_bets_list else pd.DataFrame()
        
#         if not combined_evaluated_bets_df.empty:
#             st.dataframe(combined_evaluated_bets_df[['PLAYER_NAME', 'GAME_DATE', 'Stat', 'Threshold', 'Actual Value', 'Bet Correct']])
#             total_bets = len(combined_evaluated_bets_df)
#             correct_bets = combined_evaluated_bets_df['Bet Correct'].sum()
#             correct_percentage = correct_bets / total_bets * 100 if total_bets > 0 else 0
#             st.write(f"Total Bets: {total_bets}, Correct Bets: {correct_bets}, Correct Percentage: {correct_percentage:.2f}%")
#         else:
#             st.write("No betting options generated for the selected criteria.")






#Parlay add on

# This is a simplified example. In your actual app, you'll collect bet details through the Streamlit UI.
# parlay_bets = []
# for bet in user_selected_bets:  # Assume this is a list of bets selected by the user
#     player_data = load_player_data(bet['player_name'])  # You'll need to implement this based on your app's data structure
#     probability, _, _, _, _, _, _, _ = calculate_probability(
#         player_data, bet['stat'], bet['projection'], league_std_data, n_games=10, league_std_rate=0.9, opposing_team=bet.get('opposing_team')
#     )
#     parlay_bets.append({
#         'odds': bet['odds'],
#         'probability': probability
#     })

# # Calculate parlay odds and combined probability
# parlay_odds, parlay_probability = calculate_parlay_odds(parlay_bets)

# # Calculate potential profit/loss (this part needs to be implemented based on how you handle odds and betting amounts)
# # Example: expected_profit, expected_loss, _ = calculate_bet_outcome(total_bet_amount, parlay_odds, parlay_probability)

# # Display parlay bet details
# st.write(f"Parlay Odds: {parlay_odds}, Parlay Probability: {parlay_probability*100:.2f}%")
# # st.write(f"Expected Profit: ${expected_profit:.2f}, Expected Loss: -${expected_loss:.2f}")


# Option to show full dataset
#if st.checkbox('Show Full Dataset'):
#    st.dataframe(data)


Overwriting app.py


In [13]:
%%writefile modular/betting_functions.py 
import pandas as pd
import numpy as np

#Things to consider:
#1. Calculate the probability of a player achieving a certain statistic in a game
#2. Calculate the probability of a player achieving a certain statistic against a specific team
#3. Calculate the probability of a player achieving a certain statistic in a game given the player's recent performance

#proposed changes:
#add in the opposing team as a parameter
#add in the number of games against the opposing team as a parameter
#add league std as a parameter
#add in level of competition as a warning if the opponent win rate is higher than the average win rate of this teams previous competition
#add in the number of games above projection and number of games as a parameter as a confidence rate

#***Take out the best options according to these parameters into a daily dashboard***
#record the results from this^ and see if the model is accurate


def calculate_probability(player_data, stat, projection, league_std_data, n_games=10, league_std_rate=0.9, opposing_team=None):
    # Filter out games without statistics (e.g., future games without stats yet)
    games_with_stats = player_data.dropna(subset=[stat])
    
    # Now, select the last 10 games from this filtered dataset
    last_10_games = games_with_stats.tail(n_games)
    #print(f"Last 10 games for {stat}:")
    #print(last_10_games[[stat]])
    
    # Ensure stat is used directly without alteration
    if stat not in games_with_stats.columns:
        raise KeyError(f"Statistic '{stat}' not found in player data columns.")

    # Diagnostic prints
    #print(f"Last 10 games for {stat}:")
    #print(last_10_games[[stat]])
    #print(f"Number of NaN values for {stat}: {last_10_games[stat].isna().sum()}")

    player_std = last_10_games[stat].fillna(0).std()
    #print(f"Player Std Dev for {stat}: {player_std}")

    # Initialize metrics here
    number_of_games_against_team = 0
    against_team_probability = 0
    number_of_games_above_projection_against_team = 0  # Initialize here


    if opposing_team:
        player_data.loc[:, 'OPPONENT_NAME'] = player_data['OPPONENT_NAME'].str.strip()
        
        #print(f"Opposing Team: {opposing_team}")
        #print("player_data['OPPONENT_NAME'] before=", player_data['OPPONENT_NAME'].unique())
        games_against_team = player_data[player_data['OPPONENT_NAME'] == opposing_team]
        #print("games_against_team after=", games_against_team['OPPONENT_NAME'].unique())
        #print(f"Games against {opposing_team}:")
        #print(games_against_team[[stat]])
        if not games_against_team.empty:
            games_above_projection_against_team = games_against_team[games_against_team[stat] >= projection]
            number_of_games_above_projection_against_team = len(games_above_projection_against_team)
            number_of_games_against_team = len(games_against_team)
            against_team_probability = len(games_above_projection_against_team) / number_of_games_against_team if number_of_games_against_team > 0 else 0
        #print(f"Games above projection against {opposing_team}: {number_of_games_above_projection_against_team} out of {number_of_games_against_team}")
    else:
        against_team_probability = None

    games_above_projection = last_10_games[last_10_games[stat] >= projection]
    number_of_games_above_projection = len(games_above_projection)
    number_of_games = len(last_10_games)
    probability = number_of_games_above_projection / number_of_games if number_of_games > 0 else 0

    #print(f"Games above projection: {number_of_games_above_projection} out of {number_of_games}")
    
    league_std = league_std_data[stat].iloc[0] if stat in league_std_data.columns else 0
    std_dev_comparison = player_std < league_std * league_std_rate

    #print(f"League Std Dev for {stat}: {league_std}")
    #print(f"Std Dev Comparison: {'Yes' if std_dev_comparison else 'No'}")

    return probability, against_team_probability, number_of_games_against_team, player_std, std_dev_comparison, league_std, number_of_games_above_projection, number_of_games



def calculate_bet_outcome(bet_amount, odds, probability):
    """
    Calculate expected profit or loss from a bet based on American odds.
    - For positive odds: the profit is bet_amount * (odds / 100) if win.
    - For negative odds: the profit is bet_amount / (abs(odds) / 100) if win.
    The loss is always the bet amount as you lose the stake if the bet does not win.
    """
    if odds > 0:
        # For positive odds, potential profit includes the bet amount
        potential_profit = bet_amount * (odds / 100)
    else:
        # For negative odds, potential profit is the bet amount since you need to bet more to win 100 units
        potential_profit = bet_amount / (abs(odds) / 100)

    # Expected profit considering the probability of winning
    expected_profit = potential_profit
    # No need to adjust for bet_amount for positive odds as it's considered in potential profit
    probability_weighted_to_profit = probability * potential_profit

    # Expected loss is straightforward; it's the bet amount since you lose the stake if the bet doesn't win
    expected_loss = bet_amount

    return expected_profit, expected_loss, probability_weighted_to_profit


def generate_betting_options(player_data, league_std_data, player_names, opposing_teams, n_games=10, league_std_rate=0.9, probability_high=0.9, probability_low=0.1):
    if not isinstance(player_names, list):
        player_names = [player_names]
    if opposing_teams is not None and not isinstance(opposing_teams, list):
        opposing_teams = [opposing_teams]
    """
    Generate filtered betting options based on given criteria, now including game dates.
    """
    betting_categories = {
        'PTS': np.arange(9.5, 30.5, 1),
        'AST': np.arange(2.5, 12.5, 1),
        'REB': np.arange(2.5, 12.5, 1),
        'STL': np.arange(0.5, 5.5, 1),
        'BLK': np.arange(0.5, 5.5, 1),
        'FG3M': np.arange(0.5, 5.5, 1),
    }

    results = []
    
    if all_players:
        players = player_data['PLAYER_NAME'].unique()
    else:
        players = [player_name]

    for player in players:
        player_season_data = player_data[player_data['PLAYER_NAME'] == player].copy()
        game_dates = player_season_data['GAME_DATE'].unique()

        # Corrected handling of multiple opposing teams
        for game_date in game_dates:
            game_data = player_season_data[player_season_data['GAME_DATE'] < game_date]
            if not isinstance(opposing_team, list):
                opposing_teams_temp = [opposing_team]  # Ensure opposing_teams is treated as a list
            else:
                opposing_teams_temp = opposing_team

            for opposing_team in opposing_teams_temp:
                for stat, thresholds in betting_categories.items():
                    for threshold in thresholds:
                        # Calculate probability and other metrics for the specific game
                        probability, against_team_probability, number_of_games_against_team, player_std, std_dev_comparison, league_std, number_of_games_above_projection, number_of_games = calculate_probability(
                            game_data, stat, threshold, league_std_data, n_games, league_std_rate, opposing_team)

                        if (probability > probability_high or probability < probability_low) and (player_std <= league_std * league_std_rate):
                            prob_comparison = 'Higher' if probability > probability_high else 'Lower' if probability < probability_low else 'Uncertain'
                            std_dev_comparison = 'Better than league std by at least 10%' if player_std <= league_std * league_std_rate else 'Not better than league std by at least 10%'
                            recommendation = 'Bet' if (prob_comparison == 'Higher' or prob_comparison == 'Lower') and player_std <= league_std * league_std_rate else 'Avoid'

                            result = {
                                'PLAYER_NAME': player,
                                'Stat': stat,
                                'Threshold': threshold,
                                'Probability': probability,
                                'Std Dev Comparison': std_dev_comparison,
                                'Probability comparison': prob_comparison,
                                'Recommendation based on Prob and std_dev': recommendation,
                                'Against Team Probability': against_team_probability if against_team_probability is not None else 'N/A',
                                'Games Against Team': number_of_games_against_team if number_of_games_against_team > 0 else 'N/A',
                                'GAME_DATE': game_date  # Include the game date in the results
                            }
                            results.append(result)

    results = [result for result in results if result['Recommendation based on Prob and std_dev'] == 'Bet']
                
    return pd.DataFrame(results)



def evaluate_bets(generated_bets, actual_performance):
    # Reset index to ensure it's sequential starting from 0
    generated_bets.reset_index(drop=True, inplace=True)

    # Iterate through generated bets using .iterrows() for safer access
    for index, row in generated_bets.iterrows():
        stat_column = row['Stat']
        actual_stat_row = actual_performance[
            (actual_performance['PLAYER_NAME'] == row['PLAYER_NAME']) & 
            (actual_performance['GAME_DATE'] == row['GAME_DATE'])
        ]
        
        if not actual_stat_row.empty:
            actual_value = actual_stat_row.iloc[0][stat_column]
            generated_bets.at[index, 'Actual Value'] = actual_value  # Assign actual value to the DataFrame
            
            if pd.notnull(actual_value):
                bet_correct = ((row['Probability comparison'] == 'Higher' and actual_value > row['Threshold']) or
                               (row['Probability comparison'] == 'Lower' and actual_value < row['Threshold']))
                generated_bets.at[index, 'Bet Outcome'] = bet_correct
            else:
                generated_bets.at[index, 'Bet Outcome'] = np.nan  # Mark as NaN if actual value is NaN
        else:
            # Mark as NaN if there's no matching performance data
            generated_bets.at[index, 'Actual Value'] = np.nan
            generated_bets.at[index, 'Bet Outcome'] = np.nan

    # Initialize running totals outside the loop to avoid resetting them on each iteration
    running_correct = 0
    running_incorrect = 0

    # After determining bet outcomes, calculate running totals
    for index, row in generated_bets.iterrows():
        if pd.notnull(row['Bet Outcome']):
            running_correct += int(row['Bet Outcome'] == True)
            running_incorrect += int(row['Bet Outcome'] == False)

            generated_bets.at[index, 'Running Correct'] = running_correct
            generated_bets.at[index, 'Running Incorrect'] = running_incorrect
        else:
            generated_bets.at[index, 'Running Correct'] = running_correct
            generated_bets.at[index, 'Running Incorrect'] = running_incorrect

    return generated_bets

    
def evaluate_bets_n_games_debug(generated_bets, actual_performance, n_games=10):
    # Ensure the datetime format is correct
    actual_performance['GAME_DATE'] = pd.to_datetime(actual_performance['GAME_DATE'])
    actual_performance.sort_values(by='GAME_DATE', inplace=True)
    
    evaluated_bets_list = []  # To store evaluated bets for debugging

    for player in generated_bets['PLAYER_NAME'].unique():
        print(f"\nEvaluating bets for: {player}")  # Debug: Confirm the player
        
        player_bets = generated_bets[generated_bets['PLAYER_NAME'] == player]
        player_performance = actual_performance[actual_performance['PLAYER_NAME'] == player]
        
        player_performance.sort_values(by='GAME_DATE', inplace=True)
        if len(player_performance) < n_games:
            print(f"Warning: {player} has only {len(player_performance)} games available, less than {n_games} games specified.")
        
        min_date_for_n_games = player_performance['GAME_DATE'].unique()[-n_games]
        print(f"Minimum date for the last {n_games} games: {min_date_for_n_games}")  # Debug: Check min date

        # Filter for bets and performances within the last n games
        player_bets_filtered = player_bets[player_bets['GAME_DATE'] >= min_date_for_n_games]
        player_performance_filtered = player_performance[player_performance['GAME_DATE'] >= min_date_for_n_games]

        correct_bets = 0
        incorrect_bets = 0

        for index, bet in player_bets_filtered.iterrows():
            game_date = bet['GAME_DATE']
            actual = player_performance_filtered[player_performance_filtered['GAME_DATE'] == game_date]

            if not actual.empty:
                actual_value = actual.iloc[0][bet['Stat']]
                bet_correct = ((bet['Probability comparison'] == 'Higher' and actual_value > bet['Threshold']) or
                               (bet['Probability comparison'] == 'Lower' and actual_value < bet['Threshold']))
                
                if bet_correct:
                    correct_bets += 1
                else:
                    incorrect_bets += 1
                
                evaluated_bets_list.append({**bet, 'Actual Value': actual_value, 'Bet Correct': bet_correct})
            else:
                print(f"No actual performance data for {player} on {game_date}")

        print(f"{player} - Correct Bets: {correct_bets}, Incorrect Bets: {incorrect_bets}")  # Debug: Check bet outcomes

    # Compile the debug results into a DataFrame
    evaluated_bets_df_debug = pd.DataFrame(evaluated_bets_list)
    
    if not evaluated_bets_df_debug.empty:
        final_corrects = sum(evaluated_bets_df_debug['Bet Correct'])
        overall_bets = len(evaluated_bets_df_debug)
        correct_percentage = final_corrects / overall_bets * 100
        print(f"\nFinal Correct Percentage: {correct_percentage}% ({final_corrects}/{overall_bets})")
    else:
        print("No bets evaluated. Check if the date range or player selection might be too restrictive.")

    return evaluated_bets_df_debug








#PARLAYS*********************************************************
def calculate_parlay_odds(bets):
    """
    Calculate the total odds for a parlay bet from a list of individual bets.
    Each bet in the list is a dictionary containing at least the 'odds' and 'probability' keys.
    
    Args:
    - bets (list of dicts): Each dict contains 'odds' (int or float) and 'probability' (float).

    Returns:
    - float: The total odds for the parlay bet.
    - float: The combined probability of the parlay bet winning.
    """
    total_odds = 1
    combined_probability = 1
    
    for bet in bets:
        # Convert American odds to decimal odds
        if bet['odds'] > 0:
            decimal_odds = 1 + bet['odds'] / 100
        else:
            decimal_odds = 1 - 100 / bet['odds']
        
        total_odds *= decimal_odds
        combined_probability *= bet['probability']  # Assumes independent events
    
    # Convert back to American odds if needed
    if total_odds >= 2:
        american_odds = (total_odds - 1) * 100
    else:
        american_odds = -100 / (total_odds - 1)

    return american_odds, combined_probability

#Example usage
# bets = [
#     {'odds': +150, 'probability': 0.4},
#     {'odds': -120, 'probability': 0.6},
#     {'odds': +200, 'probability': 0.3}
# ]

# parlay_odds, parlay_probability = calculate_parlay_odds(bets)
# print(f"Parlay Odds: {parlay_odds}, Parlay Probability: {parlay_probability}")






Overwriting modular/betting_functions.py


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from modular.player_game_logs import load_nba_player_game_logs, prepare_upcoming_games_data
from modular.metrics_functions import prepare_mean_std_data, prepare_league_std_data, prepare_performance_against_all_teams
from modular.betting_functions import calculate_probability, calculate_bet_outcome, generate_betting_options

# Simplified mock data loading function (no caching for debugging)
def load_data():
    data = pd.read_csv('data/player_game_logs_winr.csv')
    data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'])
    data.sort_values(by='GAME_DATE', inplace=True)
    return data

previous_games = load_data()

#pull in upcoming games to concatenate to data and input averages onto it
upcoming_games = prepare_upcoming_games_data('data/23_24_season_games.csv', 'data/player_game_logs_winr.csv', expand_with_players=True)

# Load the existing games data
previous_games = load_data()

#filter for 3-1-2024 and luka doncic and print
# selected_player = "Dejon Jarreau"
# selected_date = "2024-03-12"
# previous_games = previous_games[previous_games['GAME_DATE'] == selected_date]
# previous_games = previous_games[previous_games['PLAYER_NAME'] == selected_player]
# print(previous_games.head())

# Ensure GAME_DATE is in datetime format for comparison
upcoming_games['GAME_DATE'] = pd.to_datetime(upcoming_games['GAME_DATE'])


# Filter out upcoming games that have dates already in previous games
unique_upcoming_games = upcoming_games[~upcoming_games['GAME_DATE'].isin(previous_games['GAME_DATE'])]

# only include these columns: ['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_NAME','OPPOSING_TEAM', 'Player_ID', 'PLAYER_NAME']
unique_upcoming_games = unique_upcoming_games[['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_NAME', 'OPPONENT_NAME', 'Player_ID', 'PLAYER_NAME']]

# Concatenate the unique upcoming games to the previous games dataset
data = pd.concat([previous_games, unique_upcoming_games], ignore_index=True)

# Sort the concatenated data by GAME_DATE to maintain chronological order
data.sort_values(by='GAME_DATE', inplace=True)

# Reset the index of the concatenated DataFrame
data.reset_index(drop=True, inplace=True)

#print min and max
print(data['GAME_DATE'].min())
print(data['GAME_DATE'].max())

# selected_player = "DeJon Jarreau"
# selected_date = "2024-03-12"
# data = data[data['GAME_DATE'] == selected_date]
# players = data['PLAYER_NAME'].unique()
# print(players)
# data = data[data['PLAYER_NAME'] == selected_player]
# print(data.head())


#------------Loading data with caching---------------



c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\player_game_logs.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games.sort_values(by='DATE', inplace=True)
c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\player_game_logs.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_games['GAME_DATE'] = upcoming_games['DATE'].dt.strftime('%Y-%m-%d')
c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\player_game_logs.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'OPPONENT_ABBREVIATION', 'TEAM_NAME',
       'OPPONENT_NAME', 'TEAM_WIN_RATE', 'OPPONENT_WIN_RATE', 'home_away'],
      dtype='object')
Index(['GAME_DATE', 'MATCHUP', 'home_away', 'TEAM_ID', 'TEAM_NAME',
       'OPPONENT_NAME'],
      dtype='object')
2023-10-24 00:00:00
2024-03-25 00:00:00


In [75]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def calculate_probability(player_data, stat, projection, league_std_data, n_games=10, league_std_rate=0.9, opposing_teams=None):
    results = []
    # Ensure opposing_teams is a list for consistent processing
    opposing_teams_list = [opposing_teams] if isinstance(opposing_teams, str) else opposing_teams or ['All']
    
    for player_name in player_data['PLAYER_NAME'].unique():
        player_specific_data = player_data[player_data['PLAYER_NAME'] == player_name]

        # Iterate through the list of opposing teams or default to 'All'
        for opposing_team in opposing_teams_list:
            if opposing_team != 'All':
                # Filter games against the specific opposing team
                team_specific_data = player_specific_data[player_specific_data['OPPONENT_NAME'] == opposing_team]
            else:
                # Use all data if 'All' teams are considered
                team_specific_data = player_specific_data
            
            # Ensure there's enough data to calculate the probability
            if len(team_specific_data) < n_games:
                print(f"Not enough games for player: {player_name} against team: {opposing_team}")
                continue

            # Calculate statistics for the last n games
            recent_games = team_specific_data.sort_values('GAME_DATE', ascending=False).head(n_games)
            games_with_stat = recent_games.dropna(subset=[stat])
            num_games_with_stat_above_projection = games_with_stat[games_with_stat[stat] >= projection].shape[0]
            
            # Handle cases with no stats or all NaN values
            if games_with_stat.empty or games_with_stat[stat].std() is np.nan:
                player_std = 0
            else:
                player_std = games_with_stat[stat].std()
            
            league_std = league_std_data.get(stat, np.nan)
            std_dev_comparison = player_std < (league_std * league_std_rate)
            
            probability = num_games_with_stat_above_projection / n_games if n_games > 0 else 0
            
            # Append results for each player against each team (or 'All')
            results.append({
                'player_name': player_name,
                'opposing_team': opposing_team,
                'stat': stat,
                'projection': projection,
                'probability': probability,
                'std_dev_comparison': std_dev_comparison,
                'player_std': player_std,
                'league_std': league_std,
                'number_of_games': n_games,
                'number_of_games_above_projection': num_games_with_stat_above_projection
            })

    return pd.DataFrame(results)





def generate_betting_options(player_data, league_std_data, player_name, opposing_team, all_players=False, n_games=10, league_std_rate=0.9, probability_high=0.9, probability_low=0.1):
    """
    Generate filtered betting options based on given criteria, now including game dates.
    """
    betting_categories = {
        'PTS': np.arange(9.5, 30.5, 1),
        'AST': np.arange(2.5, 12.5, 1),
        'REB': np.arange(2.5, 12.5, 1),
        'STL': np.arange(0.5, 5.5, 1),
        'BLK': np.arange(0.5, 5.5, 1),
        'FG3M': np.arange(0.5, 5.5, 1),
    }

    results = []
    
    if all_players:
        players = player_data['PLAYER_NAME'].unique()
    else:
        players = [player_name]

    for player in players:
        # Filter player data for the specific player
        player_season_data = player_data[player_data['PLAYER_NAME'] == player].copy()

        # Get unique game dates for the player
        game_dates = player_season_data['GAME_DATE'].unique()
        #print(f"Unique game dates for player: {game_dates}")

        for game_date in game_dates:
            # Filter data for the specific game date
            #print(f"Processing game date: {game_date}")
            game_data = player_season_data[player_season_data['GAME_DATE'] < game_date]
            #print(f"Filtered game data for calculations: {game_data.shape[0]} rows")

            for stat, thresholds in betting_categories.items():
                for threshold in thresholds:
                    # Calculate probability and other metrics for the specific game
                    probability, against_team_probability, number_of_games_against_team, player_std, std_dev_comparison, league_std, number_of_games_above_projection, number_of_games = calculate_probability(
                        game_data, stat, threshold, league_std_data, n_games, league_std_rate, opposing_team)

                    if (probability > probability_high or probability < probability_low) and (player_std <= league_std * league_std_rate):
                        prob_comparison = 'Higher' if probability > probability_high else 'Lower' if probability < probability_low else 'Uncertain'
                        std_dev_comparison = 'Better than league std by at least 10%' if player_std <= league_std * league_std_rate else 'Not better than league std by at least 10%'
                        recommendation = 'Bet' if (prob_comparison == 'Higher' or prob_comparison == 'Lower') and player_std <= league_std * league_std_rate else 'Avoid'

                        result = {
                            'PLAYER_NAME': player,
                            'Stat': stat,
                            'Threshold': threshold,
                            'Probability': probability,
                            'Std Dev Comparison': std_dev_comparison,
                            'Probability comparison': prob_comparison,
                            'Recommendation based on Prob and std_dev': recommendation,
                            'Against Team Probability': against_team_probability if against_team_probability is not None else 'N/A',
                            'Games Against Team': number_of_games_against_team if number_of_games_against_team > 0 else 'N/A',
                            'GAME_DATE': game_date  # Include the game date in the results
                        }
                        results.append(result)

    results = [result for result in results if result['Recommendation based on Prob and std_dev'] == 'Bet']
                
    return pd.DataFrame(results)


import pandas as pd
import numpy as np

#old
def evaluate_bets_n_games_debug(generated_bets, actual_performance, n_games=10):
    # Ensure the datetime format is correct
    actual_performance['GAME_DATE'] = pd.to_datetime(actual_performance['GAME_DATE'])
    actual_performance.sort_values(by='GAME_DATE', inplace=True)
    
    evaluated_bets_list = []  # To store evaluated bets for debugging

    for player in generated_bets['PLAYER_NAME'].unique():
        print(f"\nEvaluating bets for: {player}")  # Debug: Confirm the player
        
        player_bets = generated_bets[generated_bets['PLAYER_NAME'] == player]
        player_performance = actual_performance[actual_performance['PLAYER_NAME'] == player]
        
        player_performance.sort_values(by='GAME_DATE', inplace=True)
        if len(player_performance) < n_games:
            print(f"Warning: {player} has only {len(player_performance)} games available, less than {n_games} games specified.")
        
        min_date_for_n_games = player_performance['GAME_DATE'].unique()[-n_games]
        print(f"Minimum date for the last {n_games} games: {min_date_for_n_games}")  # Debug: Check min date

        # Filter for bets and performances within the last n games
        player_bets_filtered = player_bets[player_bets['GAME_DATE'] >= min_date_for_n_games]
        player_performance_filtered = player_performance[player_performance['GAME_DATE'] >= min_date_for_n_games]

        correct_bets = 0
        incorrect_bets = 0

        for index, bet in player_bets_filtered.iterrows():
            game_date = bet['GAME_DATE']
            actual = player_performance_filtered[player_performance_filtered['GAME_DATE'] == game_date]

            if not actual.empty:
                actual_value = actual.iloc[0][bet['Stat']]
                bet_correct = ((bet['Probability comparison'] == 'Higher' and actual_value > bet['Threshold']) or
                               (bet['Probability comparison'] == 'Lower' and actual_value < bet['Threshold']))
                
                if bet_correct:
                    correct_bets += 1
                else:
                    incorrect_bets += 1
                
                evaluated_bets_list.append({**bet, 'Actual Value': actual_value, 'Bet Correct': bet_correct})
            else:
                print(f"No actual performance data for {player} on {game_date}")

        print(f"{player} - Correct Bets: {correct_bets}, Incorrect Bets: {incorrect_bets}")  # Debug: Check bet outcomes

    # Compile the debug results into a DataFrame
    evaluated_bets_df_debug = pd.DataFrame(evaluated_bets_list)
    
    if not evaluated_bets_df_debug.empty:
        final_corrects = sum(evaluated_bets_df_debug['Bet Correct'])
        overall_bets = len(evaluated_bets_df_debug)
        correct_percentage = final_corrects / overall_bets * 100
        print(f"\nFinal Correct Percentage: {correct_percentage}% ({final_corrects}/{overall_bets})")
    else:
        print("No bets evaluated. Check if the date range or player selection might be too restrictive.")

    return evaluated_bets_df_debug

#new
def evaluate_bets_n_games_debug(generated_bets, actual_performance, n_games=10):
    actual_performance['GAME_DATE'] = pd.to_datetime(actual_performance['GAME_DATE'])
    actual_performance.sort_values(by='GAME_DATE', inplace=True)

    # Initialize additional columns in generated_bets for evaluation metrics
    generated_bets['Correct Bets'] = 0
    generated_bets['Incorrect Bets'] = 0
    generated_bets['Total Bets Evaluated'] = 0
    generated_bets['Correct Percentage'] = 0.0

    for player in generated_bets['PLAYER_NAME'].unique():
        player_generated_bets = generated_bets[generated_bets['PLAYER_NAME'] == player].copy()
        player_actual_performance = actual_performance[actual_performance['PLAYER_NAME'] == player]

        correct_bets = 0
        incorrect_bets = 0

        for index, bet in player_generated_bets.iterrows():
            # Filtering actual performances based on the date of the bet
            relevant_actual_performances = player_actual_performance[player_actual_performance['GAME_DATE'] <= bet['GAME_DATE']].sort_values(by='GAME_DATE', ascending=False).head(n_games)
            if relevant_actual_performances.empty:
                continue  # Skip if no relevant performances found

            # Check if actual value on the game date meets the bet condition
            actual_value = relevant_actual_performances[relevant_actual_performances['GAME_DATE'] == bet['GAME_DATE']][bet['Stat']].values
            if actual_value.size > 0:
                actual_value = actual_value[0]
                bet_correct = ((bet['Probability comparison'] == 'Higher' and actual_value > bet['Threshold']) or
                               (bet['Probability comparison'] == 'Lower' and actual_value < bet['Threshold']))
                
                if bet_correct:
                    correct_bets += 1
                else:
                    incorrect_bets += 1
            else:
                print(f"No actual performance data for {player} on {bet['GAME_DATE']} for stat {bet['Stat']}")

        total_bets_evaluated = correct_bets + incorrect_bets
        correct_percentage = (correct_bets / total_bets_evaluated * 100) if total_bets_evaluated > 0 else 0

        # Update the generated_bets DataFrame with evaluation results
        generated_bets.loc[generated_bets['PLAYER_NAME'] == player, 'Correct Bets'] = correct_bets
        generated_bets.loc[generated_bets['PLAYER_NAME'] == player, 'Incorrect Bets'] = incorrect_bets
        generated_bets.loc[generated_bets['PLAYER_NAME'] == player, 'Total Bets Evaluated'] = total_bets_evaluated
        generated_bets.loc[generated_bets['PLAYER_NAME'] == player, 'Correct Percentage'] = correct_percentage

        print(f"{player} - Correct Bets: {correct_bets}, Incorrect Bets: {incorrect_bets}, Total Bets Evaluated: {total_bets_evaluated}, Correct Percentage: {correct_percentage:.2f}%")

    return generated_bets


def evaluate_bets_correctly(generated_bets, actual_performance, n_games=10):
    actual_performance['GAME_DATE'] = pd.to_datetime(actual_performance['GAME_DATE'])
    generated_bets['GAME_DATE'] = pd.to_datetime(generated_bets['GAME_DATE'])
    
    # Sort actual performance by date for sequential access
    actual_performance.sort_values(by='GAME_DATE', inplace=True)
    
    # Initialize evaluation metrics within the generated bets DataFrame
    for metric in ['Correct Bets', 'Incorrect Bets', 'Total Bets Evaluated', 'Correct Percentage']:
        generated_bets[metric] = np.nan  # Use NaN initially for clear differentiation of unprocessed data

    # Evaluate bets
    for (player, stat, game_date), group in generated_bets.groupby(['PLAYER_NAME', 'Stat', 'GAME_DATE']):
        # Find corresponding player performance up to n_games before the bet's game date
        player_performances_before_date = actual_performance[
            (actual_performance['PLAYER_NAME'] == player) & 
            (actual_performance['GAME_DATE'] < game_date)
        ].tail(n_games)

        print(f"\nEvaluating: {player}, {stat}, {game_date}, Last {n_games} games found: {len(player_performances_before_date)}")

        correct_bets = incorrect_bets = 0
        for _, bet in group.iterrows():
            for _, performance in player_performances_before_date.iterrows():
                actual_value = performance[stat]
                is_correct = ((bet['Probability comparison'] == 'Higher' and actual_value > bet['Threshold']) or
                              (bet['Probability comparison'] == 'Lower' and actual_value < bet['Threshold']))
                
                if is_correct:
                    correct_bets += 1
                else:
                    incorrect_bets += 1

        total_bets_evaluated = correct_bets + incorrect_bets
        correct_percentage = 100.0 * correct_bets / total_bets_evaluated if total_bets_evaluated > 0 else np.nan

        # Update generated_bets DataFrame with calculated metrics
        idxs = group.index
        generated_bets.loc[idxs, 'Correct Bets'] = correct_bets
        generated_bets.loc[idxs, 'Incorrect Bets'] = incorrect_bets
        generated_bets.loc[idxs, 'Total Bets Evaluated'] = total_bets_evaluated
        generated_bets.loc[idxs, 'Correct Percentage'] = correct_percentage
        
        print(f"Metrics for {player}, {stat}, {game_date}: Correct: {correct_bets}, Incorrect: {incorrect_bets}, Evaluated: {total_bets_evaluated}, % Correct: {correct_percentage}")

    return generated_bets







In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from modular.player_game_logs import load_nba_player_game_logs, prepare_upcoming_games_data
from modular.metrics_functions import prepare_mean_std_data, prepare_league_std_data, prepare_performance_against_all_teams
from modular.betting_functions import calculate_bet_outcome, generate_betting_options, evaluate_bets, calculate_probability

# Fixed values for player and date for debugging
selected_player = ["Myles Turner", "Tyrese Haliburton"]
selected_date = "2024-03-12"
end_date = "2024-03-09"
opposing_team = "Oklahoma City Thunder"  # Example opposing team for testing
stat = "PTS"
projection = 20.5  # Example projection for testing

#sort data by game date ascending
data = data.sort_values(by='GAME_DATE', ascending=True)

# Simulate selecting data for a specific player and date
# Instead of using == for comparison, use isin() for list of names
player_data_original = data[data['PLAYER_NAME'].isin(selected_player)]
player_data = player_data_original.copy()


#Testing the generate_betting_options filter-------------------------------------------------------------------------------------

# Ensure GAME_DATE is in datetime format
player_data['GAME_DATE'] = pd.to_datetime(player_data['GAME_DATE'])
player_data.sort_values(by='GAME_DATE', inplace=True)

# Filter data to include only games up to 'selected_date' for each player
player_data = player_data[player_data['GAME_DATE'] <= pd.to_datetime(selected_date)]


# Filter for selected date directly
selected_date_dt = pd.to_datetime(selected_date)

# Get today's date as a datetime object at midnight
today = datetime.now()
today = pd.to_datetime(today)
yesterday = today - timedelta(days=1)
yesterday = pd.to_datetime(yesterday)

# Filter for selected date
# If the selected date is today or in the future, filter data up to yesterday
if selected_date_dt >= today:
    print("Selected date is today or in the future.")
    player_data_filt = player_data[player_data['GAME_DATE'] <= yesterday]
# If the selected date is before today, filter for the selected day
else:
    print("Selected date is before today.")
    player_data_filt = player_data[player_data['GAME_DATE'] <= selected_date_dt]

# print("Filtered player data for selected date:", player_data_filt)

#Testing the generate_betting_options filter-------------------------------------------------------------------------------------

# Generate betting options for the selected player (assuming league_std_data is predefined or mocked similarly)
league_std_data = pd.DataFrame({
    'PTS': [10.5],
    'AST': [2.5],
    'REB': [3.5],
    'STL': [1.5],
    'BLK': [0.8],
    'FG3M': [2.0]
})

n_games = 10
league_std_rate = 0.9
probability_high = 0.9
probability_low = 0.1


# Assuming all necessary imports and functions are defined as per your script.

# Ensure the correct datetime format and sort order
print("Step 1: Data Preparation Completed")

number_of_players = len(player_data_filt['PLAYER_NAME'].unique())
# Step 2: Generate Betting Options for all historical data
betting_options_df = generate_betting_options(player_data_filt, league_std_data, selected_player, opposing_team, all_players=True, n_games=n_games, league_std_rate=league_std_rate, probability_high=probability_high, probability_low=probability_low)
print(f"Step 2: Generated {len(betting_options_df)} betting options for all historical data for {number_of_players} players:")
#print("betting options df before evaluated =", betting_options_df[['PLAYER_NAME', 'Stat', 'Threshold', 'Probability comparison', 'GAME_DATE']])

# Step 3: Filter Betting Options for the selected date
selected_date_dt = pd.to_datetime(selected_date)
print("Selected date (datetime):", selected_date_dt)
# print("betting options df game date=", betting_options_df.head())
if not betting_options_df.empty:
    betting_options_df_selected_date = betting_options_df[betting_options_df['GAME_DATE'] == selected_date_dt]
    print(betting_options_df_selected_date['GAME_DATE'].min())
    print(betting_options_df_selected_date['GAME_DATE'].max())
    number_of_players = len(betting_options_df_selected_date['PLAYER_NAME'].unique())
    print(f"Step 3: Filtered {len(betting_options_df_selected_date)} betting options for the selected date ({selected_date}) for {number_of_players} players:")
    #print(betting_options_df_selected_date[['PLAYER_NAME', 'Stat', 'Threshold', 'GAME_DATE']])

    #evaluated_bets_df = evaluate_bets_n_games_debug(betting_options_df, player_data_filt, n_games)
    evaluated_bets_df = evaluate_bets_correctly(betting_options_df, player_data_filt, n_games=10)
    #print(evaluated_bets_df.columns)
    print(evaluated_bets_df[['PLAYER_NAME', 'Stat', 'Threshold', 'Probability comparison', 'GAME_DATE', 'Correct Bets',
                              'Incorrect Bets', 'Total Bets Evaluated','Correct Percentage' ]])
    print("Step 4: Evaluated bets based on the last n games.")

    #print if incorrect bets are > 0
    print('unique incorrect bets', evaluated_bets_df['Incorrect Bets'].unique())




Selected date is before today.
Step 1: Data Preparation Completed
Step 2: Generated 1875 betting options for all historical data for 2 players:
Selected date (datetime): 2024-03-12 00:00:00
2024-03-12 00:00:00
2024-03-12 00:00:00
Step 3: Filtered 35 betting options for the selected date (2024-03-12) for 2 players:

Evaluating: Myles Turner, AST, 2023-10-30 00:00:00, Last 10 games found: 2
Metrics for Myles Turner, AST, 2023-10-30 00:00:00: Correct: 20, Incorrect: 0, Evaluated: 20, % Correct: 100.0

Evaluating: Myles Turner, AST, 2023-11-01 00:00:00, Last 10 games found: 3
Metrics for Myles Turner, AST, 2023-11-01 00:00:00: Correct: 27, Incorrect: 0, Evaluated: 27, % Correct: 100.0

Evaluating: Myles Turner, AST, 2023-11-03 00:00:00, Last 10 games found: 4
Metrics for Myles Turner, AST, 2023-11-03 00:00:00: Correct: 36, Incorrect: 0, Evaluated: 36, % Correct: 100.0

Evaluating: Myles Turner, AST, 2023-11-04 00:00:00, Last 10 games found: 5
Metrics for Myles Turner, AST, 2023-11-04 00:00

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from modular.player_game_logs import load_nba_player_game_logs, prepare_upcoming_games_data
from modular.metrics_functions import prepare_mean_std_data, prepare_league_std_data, prepare_performance_against_all_teams
from modular.betting_functions import calculate_bet_outcome, generate_betting_options, evaluate_bets, calculate_probability

# Fixed values for player and date for debugging
selected_player = ["Myles Turner", "Tyrese Haliburton"]
selected_date = "2024-03-12"
end_date = "2024-03-09"
opposing_team = "Oklahoma City Thunder"  # Example opposing team for testing
stat = "PTS"
projection = 20.5  # Example projection for testing

#sort data by game date ascending
data = data.sort_values(by='GAME_DATE', ascending=True)

# Simulate selecting data for a specific player and date
# Instead of using == for comparison, use isin() for list of names
player_data_original = data[data['PLAYER_NAME'].isin(selected_player)]
player_data = player_data_original.copy()


#Testing the generate_betting_options filter-------------------------------------------------------------------------------------

# Ensure GAME_DATE is in datetime format
player_data['GAME_DATE'] = pd.to_datetime(player_data['GAME_DATE'])
player_data.sort_values(by='GAME_DATE', inplace=True)

# Filter data to include only games up to 'selected_date' for each player
player_data = player_data[player_data['GAME_DATE'] <= pd.to_datetime(selected_date)]


# Filter for selected date directly
selected_date_dt = pd.to_datetime(selected_date)

# Get today's date as a datetime object at midnight
today = datetime.now()
today = pd.to_datetime(today)
yesterday = today - timedelta(days=1)
yesterday = pd.to_datetime(yesterday)

# Filter for selected date
# If the selected date is today or in the future, filter data up to yesterday
if selected_date_dt >= today:
    print("Selected date is today or in the future.")
    player_data_filt = player_data[player_data['GAME_DATE'] <= yesterday]
# If the selected date is before today, filter for the selected day
else:
    print("Selected date is before today.")
    player_data_filt = player_data[player_data['GAME_DATE'] <= selected_date_dt]

# print("Filtered player data for selected date:", player_data_filt)

#Testing the generate_betting_options filter-------------------------------------------------------------------------------------

# Generate betting options for the selected player (assuming league_std_data is predefined or mocked similarly)
league_std_data = pd.DataFrame({
    'PTS': [10.5],
    'AST': [2.5],
    'REB': [3.5],
    'STL': [1.5],
    'BLK': [0.8],
    'FG3M': [2.0]
})

n_games = 10
league_std_rate = 0.9
probability_high = 0.9
probability_low = 0.1


# Assuming all necessary imports and functions are defined as per your script.

# Ensure the correct datetime format and sort order
print("Step 1: Data Preparation Completed")

number_of_players = len(player_data_filt['PLAYER_NAME'].unique())
# Step 2: Generate Betting Options for all historical data
betting_options_df = generate_betting_options(player_data_filt, league_std_data, selected_player, opposing_team, all_players=True, n_games=n_games, league_std_rate=league_std_rate, probability_high=probability_high, probability_low=probability_low)
print(f"Step 2: Generated {len(betting_options_df)} betting options for all historical data for {number_of_players} players:")
# print(betting_options_df[['PLAYER_NAME', 'Stat', 'Threshold', 'Probability comparison', 'GAME_DATE']])

# Step 3: Filter Betting Options for the selected date
selected_date_dt = pd.to_datetime(selected_date)
print("Selected date (datetime):", selected_date_dt)
# print("betting options df game date=", betting_options_df.head())
if not betting_options_df.empty:
    betting_options_df_selected_date = betting_options_df[betting_options_df['GAME_DATE'] == selected_date_dt]
    number_of_players = len(betting_options_df_selected_date['PLAYER_NAME'].unique())
    print(f"Step 3: Filtered {len(betting_options_df_selected_date)} betting options for the selected date ({selected_date}) for {number_of_players} players:")
    #print(betting_options_df_selected_date[['PLAYER_NAME', 'Stat', 'Threshold', 'GAME_DATE']])

    #Step 4: Evaluate Bets for an overall evaluation based on the last n games
    # Ensure the correct datetime format and sort order
    player_data_filt['GAME_DATE'] = pd.to_datetime(player_data_filt['GAME_DATE'])
    player_data_filt.sort_values(by='GAME_DATE', inplace=True)
    unique_dates = betting_options_df['GAME_DATE'].unique()
    if len(unique_dates) > n_games:
        print("Unique dates are greater than n_games")
        print("min and max date unique dates=", unique_dates.min(), unique_dates.max())
        unique_dates_n_games = unique_dates[-n_games:]
        print("min and max date unique n games=", unique_dates_n_games.min(), unique_dates_n_games.max())
        betting_options_df_n_games = betting_options_df[betting_options_df['GAME_DATE'].isin(unique_dates_n_games)]
    else:
        betting_options_df_n_games = betting_options_df


    print("betting options columns=", betting_options_df.columns)
    print("player data filt columns=", player_data_filt.columns)
    evaluated_bets_df = evaluate_bets(betting_options_df_n_games, player_data_filt)#, n_games)
    print("Step 4: Evaluated bets based on the last n games.")

    # Debugging prints for evaluated bets
    print(f"Evaluated Bets DataFrame has {len(evaluated_bets_df)} rows after evaluation.")

    # Step 5: Display Outputs
    # Filtering out NaN values from evaluated bets
    evaluated_bets_df_cleaned = evaluated_bets_df.dropna(subset=['Bet Outcome'])
    print(f"Step 5: After cleaning, {len(evaluated_bets_df_cleaned)} evaluated bets remain.")
    #print(evaluated_bets_df_cleaned[['PLAYER_NAME', 'Stat', 'Threshold', 'Actual Value', 'Bet Outcome', 'Running Correct', 'Running Incorrect']])

    #print min and max date
    min_date = evaluated_bets_df_cleaned['GAME_DATE'].min()
    max_date = evaluated_bets_df_cleaned['GAME_DATE'].max()
    print("evaluted bets min date =", min_date)
    print("evaluted bets max date =", max_date)

    # Note: Modify print statements as per your debugging needs


    # print("evaluated_bets_df =", evaluated_bets_df.head())
    # print(evaluated_bets_df.columns)

    #filter out NaN 
    # print(len(evaluated_bets_df))
    evaluated_bets_df = evaluated_bets_df.dropna(subset=['Bet Outcome'])
    # print(len(evaluated_bets_df))
    #filter for bet_outcome = 0 ?
    evaluated_bets_df = evaluated_bets_df[evaluated_bets_df['Running Incorrect'] >= 1]

    #print(evaluated_bets_df.columns)
    #print(evaluated_bets_df[['PLAYER_NAME', 'Stat', 'Threshold', 'Probability comparison', 'Actual Value', 'Bet Outcome', 'Running Correct', 'Running Incorrect']])

    # Display the last values of Running Correct and Running Incorrect
    if not evaluated_bets_df.empty:
        final_corrects = evaluated_bets_df.iloc[-1]['Running Correct']
        final_incorrects = evaluated_bets_df.iloc[-1]['Running Incorrect']
        overall_bets = final_corrects + final_incorrects
        print(f"Final Corrects: {final_corrects}, Final Incorrects: {final_incorrects}")
    else:
        print("No bets evaluated.")
    
    #Display Percentage of correct bets through Running Correct and Running Incorrect
    if final_corrects + final_incorrects > 0:
        correct_percentage = final_corrects / overall_bets
        print(f"Percentage of correct bets: {correct_percentage*100:.2f}% out of {overall_bets} chances over the last {n_games} games ({min_date} to {max_date})")
    else:
        print("No bets evaluated yet.")


Selected date is before today.
Step 1: Data Preparation Completed
Step 2: Generated 1875 betting options for all historical data for 2 players:
Selected date (datetime): 2024-03-12 00:00:00
Step 3: Filtered 35 betting options for the selected date (2024-03-12) for 2 players:
Unique dates are greater than n_games
min and max date unique dates= 2023-10-30 00:00:00 2024-03-12 00:00:00
min and max date unique n games= 2023-12-18 00:00:00 2024-03-12 00:00:00
betting options columns= Index(['PLAYER_NAME', 'Stat', 'Threshold', 'Probability', 'Std Dev Comparison',
       'Probability comparison', 'Recommendation based on Prob and std_dev',
       'Against Team Probability', 'Games Against Team', 'GAME_DATE'],
      dtype='object')
player data filt columns= Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 

c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\betting_functions.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_bets.at[index, 'Actual Value'] = actual_value  # Assign actual value to the DataFrame
c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\betting_functions.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_bets.at[index, 'Bet Outcome'] = bet_correct


Step 4: Evaluated bets based on the last n games.
Evaluated Bets DataFrame has 263 rows after evaluation.
Step 5: After cleaning, 263 evaluated bets remain.
evaluted bets min date = 2023-12-18 00:00:00
evaluted bets max date = 2024-03-12 00:00:00
Final Corrects: 248.0, Final Incorrects: 15.0
Percentage of correct bets: 94.30% out of 263.0 chances over the last 10 games (2023-12-18 00:00:00 to 2024-03-12 00:00:00)


c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\betting_functions.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_bets.at[index, 'Running Correct'] = running_correct
c:\Users\ghadf\vscode_projects\nba_player_analysis\modular\betting_functions.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_bets.at[index, 'Running Incorrect'] = running_incorrect


In [5]:
pd.options.mode.chained_assignment = None  # Disable the warning
